In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

! pip install beautifulsoup4
from bs4 import BeautifulSoup
from urllib import request
html= request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(html, 'lxml')

table = soup.find_all('table')[0]
raw_data=pd.read_html(str(table))[0]

raw_data.columns = raw_data.iloc[0]
df_postcode = raw_data.drop(0)

df_postcode = df_postcode[~df_postcode['Borough'].str.contains('Not assigned')]
df_postcode.reset_index(drop = True, inplace = True)

df_postcode.loc[df_postcode.Neighbourhood == "Not assigned", 'Neighbourhood'] = df_postcode['Borough']

Nei=df_postcode.groupby(by='Postcode').apply(lambda x:','.join(x['Neighbourhood']))
df_Nei=Nei.to_frame('Neighourhood')
df_Nei.reset_index(inplace=True)

df_bor = df_postcode.drop_duplicates(['Postcode'])
df_bor.reindex()
df_bor.drop(['Neighbourhood'], axis =1, inplace = True)

df_postcode_new = pd.merge(df_bor, df_Nei )

Solving environment: done

# All requested packages already installed.

Libraries imported.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_postcode_2 = pd.merge(df_postcode_new, df_geo)
df_postcode_2.head()

,Postcode,Borough,Neighourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
